In [364]:
import glob
import itertools
import json
import os
import pandas

pandas.options.display.max_rows = None

In [365]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return int(fragments[2])

In [366]:
# path = "ExperimentData//"
# path = "SampleSessionLayerableOnly//"
# path = "1652473795203//"
# path = "1652484257598//"
# path = "31/PersonalAR_2//"

session_paths = glob.glob("DataCollection-20220517T162747Z-001/DataCollection/47/PersonalAR/*")
print(session_paths)
dir_list = []
for session_path in session_paths:
    dir_list.extend(os.listdir(session_path))

dir_list.sort(key=parseFileNumber)

print("Using session paths: ")
print(session_paths)
# print("Files and directories in '", path, "' :")
 
# prints all files
# print ("Found following session files: ")
# print(dir_list)
# print(dir_list)


['DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653602626391', 'DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653604023476', 'DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653604528225', 'DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653604610535']
Using session paths: 
['DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653602626391', 'DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653604023476', 'DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653604528225', 'DataCollection-20220517T162747Z-001/DataCollection/84/PersonalAR\\1653604610535']


In [367]:
appEvents = []
gestureEvents = []
codesEvents = []
experimentEvents = []

def parseData(json_data):
    frames_list = []
    frames = json_data["frames"]
    for frame in frames:
        frameData = {
            "frameNum": int,
            "timestamp": float,
            "systemTime": str,
            "unixTime": float,
            "gazeValid": bool,

            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},

            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},

            "hand":{},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "openApps": [],
            "appEvents": [],
            "placedObjects": [],
            "anchorEvents":[],
            "codesEvents":[],
            "gestureEvents":[],
            "IsLayerableMode": bool,
            "experimentEvents": []
        }

        appEvent = frame["appEvents"]
        codeEvent = frame["codesEvents"]
        experimentEvent = frame["experimentEvents"]
        gestureEvent = frame["gestureEvents"]

        frameData["frameNum"] = frame["frameNum"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["systemTime"] = frame["systemTime"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["gazeValid"] = frame["gazeValid"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]

        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]

        frameData["hand"] = frame["hand"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["openApps"] = frame["openApps"]
        frameData["appEvents"] = appEvent
        frameData["placedObjects"] = frame["placedObjects"]
        frameData["anchorEvents"] = frame["anchorEvents"]
        frameData["codesEvents"] = codeEvent
        frameData["gestureEvents"] = gestureEvent
        frameData["IsLayerableMode"] = frame["IsLayerableMode"]
        frameData["experimentEvents"] = experimentEvent
        frames_list.append(frameData)
        if appEvent:
            appEvents.append(appEvent)
        if codeEvent:
            codesEvents.append(codeEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
        if gestureEvent:
            gestureEvents.append(gestureEvent)
    return frames_list


In [368]:
frames = []

def extract_frames(session_path):
    session_dir_list = os.listdir(session_path)
    extracted_frames = []
    for filename in session_dir_list:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            #print(filename)
            firstFileFlag = False
            if "Session_0" in filename:
                firstFileFlag = True
            with open(os.path.join(session_path, filename), 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    #data.append(json_data["obj"]["sessionRecording"])
                    session_frames = parseData(json_data["obj"]["sessionRecording"])
                    extracted_frames.append(session_frames)
                else:
                    #data.append(json.load(f))
                    session_frames = parseData(json_data)
                    extracted_frames.append(session_frames)
    return extracted_frames

for session_path in session_paths:
    frames.extend(extract_frames(session_path))

In [369]:
frames
framesUnpacked = list(itertools.chain(*frames))
framesDf = pandas.DataFrame(framesUnpacked)

framesDf = framesDf.sort_values(by='timestamp')
framesDf.head(1)
framesDf.head(2)

,frameNum,timestamp,systemTime,unixTime,gazeValid,hPos,hDir,hRot,hAngl,gazeOrigin,...,rightHandRay,leftHandRay,openApps,appEvents,placedObjects,anchorEvents,codesEvents,gestureEvents,IsLayerableMode,experimentEvents
4711,1,2130,15-27-05-60,1653604025606,False,"{'x': -0.14367735385894775, 'y': -0.0040016127...","{'x': 0.3431665003299713, 'y': -0.029299274086...","{'x': 0.018977556377649307, 'y': 0.17387093603...","{'x': 1.6789649724960327, 'y': 20.078901290893...","{'x': -0.1279430091381073, 'y': -0.00421984773...",...,"{'x': 0.0, 'y': 0.0, 'z': 0.0}","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],[],"[coffee maker, window blinds, thermostat, prin...","[{'unixTime': 1653604023626, 'systemTime': '15...",[],[],True,[]
4712,2,2279,15-27-05-75,1653604025756,False,"{'x': -0.14454059302806854, 'y': 0.00400076573...","{'x': 0.340556800365448, 'y': 0.05283773690462...","{'x': -0.022684656083583832, 'y': 0.1735406219...","{'x': 356.9712219238281, 'y': 19.9398326873779...","{'x': -0.1423298716545105, 'y': 0.000561410095...",...,"{'x': 0.0, 'y': 0.0, 'z': 0.0}","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],[],"[coffee maker, window blinds, thermostat, prin...",[],[],[],True,[]


In [370]:
layerableOpenApps = framesDf.loc[(framesDf['IsLayerableMode'] == True)]
immersiveOpenApps = framesDf.loc[(framesDf['IsLayerableMode'] == False)]

immersiveOpenAppsDf = pandas.DataFrame(immersiveOpenApps["openApps"])
layerableOpenAppsDf = pandas.DataFrame(layerableOpenApps["openApps"])

immersiveOpenAppsDf.head(2)

for im in immersiveOpenAppsDf["openApps"]:
    if len(im) > 1:
        print(im)

avg = 0
count = 0
max = 0
min = 6
for lay in layerableOpenAppsDf["openApps"]:
    numAppOpen = len(lay)
    avg = numAppOpen + avg
    count = count + 1
    if(numAppOpen > max):
        max = numAppOpen
    if(numAppOpen < min):
        min = numAppOpen
    

print(avg/count)
print(max)
print(min)

['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['Atlas', 'AnnotationsMenu']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
['AnnotationsMenu', 'Item Inspector']
3.8104890148830615
5
0


In [371]:
gestureEvents
gestureEventsUnpacked = list(itertools.chain(*gestureEvents))
gestureEventsDf = pandas.DataFrame(gestureEventsUnpacked)

selectGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Select"]
manipulateGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Manipulate Action"]
navigateGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Navigation Action"]
holdGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Hold Action"]

In [372]:
leftRay = framesDf.loc[framesDf["leftHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]
rightRay = framesDf.loc[framesDf["rightHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]

In [373]:
#experimentEvents[0][0]
experimentEvents
pinCodes = experimentEvents[0][0]["pinCodes"]

for exper in experimentEvents:
    for exp in exper:
        if exp["eventName"] == "EndTask":
            print(exp)


{'unixTime': 1653603563403, 'systemTime': '15-19-23-40', 'userID': '1653602626391', 'sessionType': 'Mode A', 'eventName': 'EndTask', 'pinRandomSeed': 9549, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 2}
{'unixTime': 1653604463473, 'systemTime': '15-34-23-47', 'userID': '1653604023476', 'sessionType': 'Mode B', 'eventName': 'EndTask', 'pinRandomSeed': 2264, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 8}
{'unixTime': 1653605038313, 'systemTime': '15-43-58-31', 'userID': '1653604610535', 'sessionType': 'Mode A', 'eventName': 'EndTask', 'pinRandomSeed': 5395, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 13}
{'unixTime': 1653605486150, 'systemTime': '15-51-26-14', 'userID': '1653604610535', 'sessionType': 'Mode B', 'eventName': 'EndTask', 'pinRandomSeed': 5395, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 9}


In [374]:
experimentEvents
experimentsUnpacked = list(itertools.chain(*experimentEvents))

experimentsDf = pandas.DataFrame(experimentsUnpacked).sort_values(by="unixTime")
experimentsDf.tail(24)



,unixTime,systemTime,userID,sessionType,eventName,pinRandomSeed,pinNumCodes,pinCodeLength,pinNumPieces,pinCodes,score
0,1653603143391,15-12-23-39,1653602626391,Mode A,StartTask,9549,200,6,3,"[{'Value': 429011, 'Label': 'GD', 'Pieces': [{...",0
1,1653603563403,15-19-23-40,1653602626391,Mode A,EndTask,9549,200,6,3,[],2
2,1653604043446,15-27-23-44,1653604023476,Mode B,StartTask,2264,200,6,3,"[{'Value': 547438, 'Label': 'KD', 'Pieces': [{...",0
3,1653604463473,15-34-23-47,1653604023476,Mode B,EndTask,2264,200,6,3,[],8
4,1653604618316,15-36-58-31,1653604610535,Mode A,StartTask,5395,200,6,3,"[{'Value': 289731, 'Label': 'LS', 'Pieces': [{...",0
5,1653605038313,15-43-58-31,1653604610535,Mode A,EndTask,5395,200,6,3,[],13
6,1653605066042,15-44-26-04,1653604610535,Mode B,StartTask,5395,200,6,3,"[{'Value': 289731, 'Label': 'LS', 'Pieces': [{...",0
7,1653605486150,15-51-26-14,1653604610535,Mode B,EndTask,5395,200,6,3,[],9


In [375]:
#check experiment events
immersiveStarts = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'StartTask')]
immersiveEnds = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'EndTask')]
immersiveStarts.reset_index(drop=True, inplace=True)
immersiveEnds.reset_index(drop=True, inplace=True)
immersiveEnds = immersiveEnds.assign(timeDifference = immersiveEnds['unixTime'].sub(immersiveStarts['unixTime'], fill_value=0))
immersiveTime = immersiveEnds['timeDifference'].sum() / 1000 / 60

layerableStarts = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode A') & (experimentsDf['eventName'] == 'StartTask')]
layerableEnds = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode A') & (experimentsDf['eventName'] == 'EndTask')]
layerableStarts.reset_index(drop=True, inplace=True)
layerableEnds.reset_index(drop=True, inplace=True)
layerableEnds = layerableEnds.assign(timeDifference = layerableEnds['unixTime'].sub(layerableStarts['unixTime'], fill_value=0))
layerableTime = layerableEnds['timeDifference'].sum() / 1000 / 60

experimentStarts = experimentsDf.loc[experimentsDf['eventName'] == 'StartTask']
experimentStops = experimentsDf.loc[experimentsDf['eventName'] == 'EndTask']

immersiveScore = experimentStops.loc[experimentStops['sessionType'] == 'Mode B']['score'].to_numpy()
layerableScore = experimentStops.loc[experimentStops['sessionType'] == 'Mode A']['score'].to_numpy()

In [376]:
immersiveStartTimes = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'StartTask')]

In [377]:
codesEvents
codesUnpacked = list(itertools.chain(*codesEvents))

codesDf = pandas.DataFrame(codesUnpacked).sort_values(by="unixTime")
# codesDf.sort_values(by="unixTime")

In [378]:
successfulCodes = codesDf.loc[codesDf['success'] == True]
# successfulCodes.sort_values(by="unixTime")

In [379]:
appEvents
appsUnpacked = list(itertools.chain(*appEvents))
appsDf = pandas.DataFrame(appsUnpacked)
appsDf = appsDf.sort_values(by='eventTime')
#appsDf

In [380]:
#check time spent in each app

tutorialEvents = appsDf[appsDf['name'] == 'Tutorial']
annotationEvents = appsDf[appsDf['name'] == 'AnnotationsMenu']
codeEntryEvents = appsDf[appsDf['name'] == 'Code Entry']
itemInspectorEvents = appsDf[appsDf['name'] == 'Item Inspector']
deviceGroupsEvents = appsDf[appsDf['name'] == 'Device Groups']
atlasEvents = appsDf[appsDf['name'] == 'Atlas']

annotationStarts = annotationEvents[annotationEvents['activity'] == 'Start']
annotationStops = annotationEvents[annotationEvents['activity'] == 'Stop']
annotationStarts.reset_index(drop=True, inplace=True)
annotationStops.reset_index(drop=True, inplace=True)
annotationStops = annotationStops.assign(timeDifference = annotationStops['unixTime'].sub(annotationStarts['unixTime'], fill_value=0))
annotationTime = annotationStops['timeDifference'].sum() / 1000 / 60
print("time spent in annotation menu: " + str(annotationTime))

tutorialStarts = tutorialEvents[tutorialEvents['activity'] == 'Start']
tutorialStops = tutorialEvents[tutorialEvents['activity'] == 'Stop']
tutorialStops.reset_index(drop=True, inplace = True)
tutorialStarts.reset_index(drop = True, inplace = True)
tutorialStops = tutorialStops.assign(timeDifference = tutorialStops['unixTime'].sub(tutorialStarts['unixTime'],fill_value=0))
tutorialTime = tutorialStops['timeDifference'].sum() / 1000 / 60
print("time spent in tutorial: " + str(tutorialTime))

codeEntryStarts = codeEntryEvents[codeEntryEvents['activity'] == 'Start']
codeEntryStops = codeEntryEvents[codeEntryEvents['activity'] == 'Stop']
codeEntryStarts.reset_index(drop=True, inplace=True)
codeEntryStops.reset_index(drop=True, inplace=True)
codeEntryStops = codeEntryStops.assign(timeDifference = codeEntryStops['unixTime'].sub(codeEntryStarts['unixTime'], fill_value=0))
codeEntryTime = codeEntryStops['timeDifference'].sum() / 1000 / 60
print("time spent in code entry: " + str(codeEntryTime))

itemInspectorStarts = itemInspectorEvents[itemInspectorEvents['activity'] == 'Start']
itemInspectorStops = itemInspectorEvents[itemInspectorEvents['activity'] == 'Stop']
itemInspectorStarts.reset_index(drop=True, inplace=True)
itemInspectorStops.reset_index(drop=True, inplace=True)
itemInspectorStops = itemInspectorStops.assign(timeDifference = itemInspectorStops['unixTime'].sub(itemInspectorStarts['unixTime'], fill_value=0))
itemInspectorTime = itemInspectorStops['timeDifference'].sum() / 1000 / 60
print("time spent in item inspector: " + str(itemInspectorTime))

deviceGroupsStarts = deviceGroupsEvents[deviceGroupsEvents['activity'] == 'Start']
deviceGroupsStops = deviceGroupsEvents[deviceGroupsEvents['activity'] == 'Stop']
deviceGroupsStarts.reset_index(drop=True, inplace=True)
deviceGroupsStops.reset_index(drop=True, inplace=True)
deviceGroupsStops = deviceGroupsStops.assign(timeDifference = deviceGroupsStops['unixTime'].sub(deviceGroupsStarts['unixTime'], fill_value = 0))
deviceGroupsTime = deviceGroupsStops['timeDifference'].sum() / 1000 / 60
print("time spent in device groups: " + str(deviceGroupsTime))

atlasStarts = atlasEvents[atlasEvents['activity'] == 'Start']
atlasStops = atlasEvents[atlasEvents['activity'] == 'Stop']
atlasStarts.reset_index(drop=True, inplace=True)
atlasStops.reset_index(drop=True, inplace=True)
atlasStops = atlasStops.assign(timeDifference = atlasStops['unixTime'].sub(atlasStarts['unixTime'], fill_value=0))
atlasTime = atlasStops['timeDifference'].sum() / 1000 / 60
print("time spent in atlas: " + str(atlasTime))

print("\ntime spent in immersive mode: " + str(immersiveTime))
print("time spent in layerable mode: " + str(layerableTime))




time spent in annotation menu: 0.11616666666666667
time spent in tutorial: 8.787816666666666
time spent in code entry: 75.91146666666667
time spent in item inspector: 56.29208333333334
time spent in device groups: 51.27133333333334
time spent in atlas: 55.99328333333334

time spent in immersive mode: 14.00225
time spent in layerable mode: 14.00015


In [381]:

print("number of experiment starts: " + str(experimentStarts.shape[0]))
print("number of experiment stops: " + str(experimentStops.shape[0]))

number of experiment starts: 4
number of experiment stops: 4


In [382]:
print("number of correct codes entered: " + str(successfulCodes.shape[0]))
for score in immersiveScore:
    print("number of correct codes entered in immersive mode: " + str(score))
for score in layerableScore:
    print("number of correct codes entered in layerable mode: " + str(score))

number of correct codes entered: 33
number of correct codes entered in immersive mode: 8
number of correct codes entered in immersive mode: 9
number of correct codes entered in layerable mode: 2
number of correct codes entered in layerable mode: 13


In [383]:
placedObject = framesDf.loc[:,"placedObjects"]

for p in placedObject:
    if( len(p) < 15):
        print(len(p))

In [384]:
print("number of times left ray was tracked: " + str(leftRay.shape[0]))
print("number of times right ray was tracked: " + str(rightRay.shape[0]))

print("\nnumber of select gestures: " + str(selectGesture.shape[0]))
print("number of manipulate gestures: " + str(manipulateGesture.shape[0]))
print("number of navigate gestures: " + str(navigateGesture.shape[0]))
print("number of hold gestures: " + str(holdGesture.shape[0]))

number of times left ray was tracked: 1325
number of times right ray was tracked: 5203

number of select gestures: 8
number of manipulate gestures: 80
number of navigate gestures: 79
number of hold gestures: 0
